In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.preprocessing as preprocessing
from collections import Counter
import random
import import_ipynb
import data_preprocessing

importing Jupyter notebook from data_preprocessing.ipynb


In [2]:
def get_contexts(batch,i,window_size):
    n=window_size//2
    return list(set(batch[max(0,i-n):i]+batch[i+1:min(len(batch),i+n+1)]))

#generator for batches
def get_batch_sg(words,batch_size,window_size):
    n_batches=len(words)//batch_size
    words=words[:n_batches*batch_size]
    for batch_start in range(0,len(words),batch_size):
        batch=words[batch_start:batch_start+batch_size]
        x,y=[],[]
        for i in range(len(batch)):
            center=batch[i]
            contexts=get_contexts(batch,i,window_size)
            y.extend(contexts)
            x.extend([center]*len(contexts))
        yield x,y 
def train_sg(int_words,vocab_to_int):
    vocab_size=len(vocab_to_int)
    # hyperparameters
    epochs=300
    batch_size=100
    window_size=5
    word_dimension=300
    n_samples=10
    
    inputs=tf.placeholder(tf.int32,[None],name='inputs') # size is variable , inputs are indexes of words in the batch
    labels=tf.placeholder(tf.int32,[None,None],name='labels')
    with tf.variable_scope("skip_gram"):
        embedding_V=tf.Variable(tf.random_uniform((vocab_size,word_dimension),-1,1))
        embed=tf.nn.embedding_lookup(embedding_V,inputs) # chooses the given rows
        embedding_U=tf.Variable(tf.random_normal((vocab_size,word_dimension)))
        softmax_biases=tf.Variable(tf.zeros(vocab_size))
        #loss with negative_sampling
        loss=tf.nn.sampled_softmax_loss(weights=embedding_U,biases=softmax_biases,inputs=embed,labels=labels,num_sampled=n_samples,num_classes=vocab_size)
        cost=tf.reduce_mean(loss)
        optimizer=tf.train.AdamOptimizer().minimize(cost)
    
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(epochs):
            cost_value=0
            batch_generator=get_batch_sg(int_words,batch_size,window_size)
            for x,y in batch_generator:
                feed_dict={inputs:x,labels:np.array(y)[:,None]}  #labels:np.array(y)[:,None] adds a dimmension, is like squeeze(1)
                _,cost_val=sess.run([optimizer,cost],feed_dict)
                cost_value+=cost_val
            print('epoch_{}'.format(epoch),'cost_value: ',cost_value)
    return embedding_V,embedding_U

if __name__=='__main__':
    int_words,vocab_to_int=data_preprocessing.get_int_words()
    center_embeds_sg,context_embeds_sg=train_sg(int_words,vocab_to_int)
    word_embeddings=center_embeds_sg+context_embeds_sg
    

epoch_0 cost_value:  326.296462059021
epoch_1 cost_value:  313.9303913116455
epoch_2 cost_value:  302.6006898880005
epoch_3 cost_value:  300.20207118988037
epoch_4 cost_value:  287.6784973144531
epoch_5 cost_value:  287.12732791900635
epoch_6 cost_value:  273.1272792816162
epoch_7 cost_value:  263.6047639846802
epoch_8 cost_value:  263.5993061065674
epoch_9 cost_value:  251.07079315185547
epoch_10 cost_value:  250.7352590560913
epoch_11 cost_value:  229.14344310760498
epoch_12 cost_value:  237.32798767089844
epoch_13 cost_value:  228.4108428955078
epoch_14 cost_value:  214.32512283325195
epoch_15 cost_value:  217.5801820755005
epoch_16 cost_value:  211.8976092338562
epoch_17 cost_value:  204.04852151870728
epoch_18 cost_value:  204.49097347259521
epoch_19 cost_value:  192.386474609375
epoch_20 cost_value:  189.14686250686646
epoch_21 cost_value:  190.29957962036133
epoch_22 cost_value:  184.62763452529907
epoch_23 cost_value:  176.06454515457153
epoch_24 cost_value:  166.40079259872437

KeyboardInterrupt: 